In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
# if data not saved as csv, run this
import os
if not os.path.exists('data/merged_titles_labels.csv'):
    df1 = pd.read_csv('../eda/small1/labeled.csv')
    df2 = pd.read_csv('../eda/small2/labeled.csv')
    df3 = pd.read_csv('../eda/small3/labeled.csv')
    df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
    df.to_csv('data/merged_titles_labels.csv', index=False)
    df.head()
else:
    df = pd.read_csv('data/merged_titles_labels.csv')
df.head()

,title,is_clickbait,text,dataset,sample
0,"17 Times ""Game Of Thrones"" Matched Up Perfectl...",1,NaN,clickbait-dataset,train
1,"Facing Texas in Tar Heels Territory, Duke Find...",0,NaN,clickbait-dataset,train
2,Several GOP States Move to Block Their Own Fun...,0,A number of GOP states have moved to introduce...,clickbait-news-detection,train
3,"For Hillary Clinton and John Kerry, Divergent ...",0,"WASHINGTON — Early in 2011, after a hectic ...",fake-news,train
4,"Wikinews interviews Kevin Baugh, president of ...",0,NaN,clickbait-dataset,train


In [60]:
%run functions.py
if not os.path.exists('data/preprocessed_titles_labels.pkl'):
    df = preprocess_title(df, verbose=True)
    df.to_pickle('data/preprocessed_titles_labels.pkl') 

else:
    df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


0        17 Times "Game Of Thrones" Matched Up Perfectl...
1        Facing Texas in Tar Heels Territory, Duke Find...
2        Several GOP States Move to Block Their Own Fun...
3        For Hillary Clinton and John Kerry, Divergent ...
4        Wikinews interviews Kevin Baugh, president of ...
                               ...                        
74072    The Famous ‘Mary Tyler Moore Show’ House Is Up...
74073    Hillary Literally Became the Definition of ‘Pa...
74074               NASA says worker tried to sabotage ISS
74075    SC Hears Nirbhaya's Rapists Death Penalty Appe...
74076    Flash Mob of Black Teens Assault Whites, Media...
Name: title, Length: 74077, dtype: object
Removing numbers and replacing with words...
0        seventeen Times "Game Of Thrones" Matched Up P...
1        Facing Texas in Tar Heels Territory, Duke Find...
2        Several GOP States Move to Block Their Own Fun...
3        For Hillary Clinton and John Kerry, Divergent ...
4        Wikinews interviews

D:\Inzynierka\pracaInzynierska\modelling\functions.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: [word for word in x if word not in stop_words])
D:\Inzynierka\pracaInzynierska\modelling\functions.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,is_clickbait,text,dataset,sample
0,"[seventeen, time, game, throne, matched, perfe...",1,NaN,clickbait-dataset,train
1,"[facing, texas, tar, heel, territory, duke, fi...",0,NaN,clickbait-dataset,train
2,"[several, gop, state, move, block, funding, sa...",0,A number of GOP states have moved to introduce...,clickbait-news-detection,train
3,"[hillary, clinton, john, kerry, divergent, pat...",0,"WASHINGTON — Early in 2011, after a hectic ...",fake-news,train
4,"[wikinews, interview, kevin, baugh, president,...",0,NaN,clickbait-dataset,train


In [61]:
df['title'][0]

['seventeen',
 'time',
 'game',
 'throne',
 'matched',
 'perfectly',
 'park',
 'rec',
 'quote']

In [62]:
df['sample'].value_counts()

sample
train    66664
test      7407
Name: count, dtype: int64

In [63]:
df_train = df[df['sample']=='train'].reset_index(drop=True)

In [64]:
df_train['sample'].value_counts()

sample
train    66664
Name: count, dtype: int64

In [65]:
from tqdm.notebook import tqdm
import time

EPOCHS = 500
WORKERS = 16
MIN_COUNT = 1


# train many word2vec models with diferent VECTOR_SIZE and WINDOW

VECTOR_SIZEs = [
    # 10,
    # 20,
    # 50,
    100, 
    250, 
    # 500, 
    # 1000, 
   
]

WINDOWs = [
    3, 
    4, 
    5, 
    6, 
    7, 
    8
]
SGs = [0, 1]


##################################################
# # --uncomment for sample model training--
# EPOCHS = 200
# VECTOR_SIZEs = [500]
# WINDOWs = [4]
# SGs = [0]
##################################################


print('Start training')
# sleep 200 ms
time.sleep(0.2)

for VECTOR_SIZE in tqdm(VECTOR_SIZEs):
    print(f'Current VECTOR_SIZE: {VECTOR_SIZE}')
    for WINDOW in tqdm(WINDOWs, desc=f'WINDOW'):
        for sg in tqdm(SGs, desc=f'SG'):
            # check if model already trained
            if os.path.exists(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model'):
                print(f'word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model already exists')
            else:
                model = Word2Vec(df_train['title'], vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, sg=sg)
                model.train(df_train['title'], total_examples=len(df_train['title']), epochs=EPOCHS)
                model.save(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model')

Start training


  0%|          | 0/2 [00:00<?, ?it/s]

Current VECTOR_SIZE: 100


WINDOW:   0%|          | 0/6 [00:00<?, ?it/s]

SG:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.wv.most_similar('trump')

[('donald', 0.7094179391860962),
 (' ', 0.45392268896102905),
 ('donaldbreaux', 0.4396342933177948),
 ('breitbart', 0.4336661994457245),
 ('mcgahn', 0.41913774609565735),
 ('sketchy', 0.4158231317996979),
 ('mexicano', 0.41405946016311646),
 ('historischer', 0.408134788274765),
 ('infowarriors', 0.4076419472694397),
 ('trumpprotest', 0.40638548135757446)]

In [ ]:
# save vocabulary
with open('data/vocabulary.txt', 'w') as f:
    for word in model.wv.index_to_key:
        f.write(word + '\n')